In [4]:
from molytica_m.data_tools import PPI_interactome_tools
from molytica_m.data_tools import alpha_fold_tools

2023-12-12 19:43:12.293820: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-12 19:43:12.293853: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-12 19:43:12.294223: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-12 19:43:12.346797: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-12 19:43:13.157845: W tensorflow/compiler/

cuda


In [5]:
af_uniprots = alpha_fold_tools.get_alphafold_uniprot_ids()
PPI_interactome_tools.get_ppi_prob(af_uniprots[-1], af_uniprots[-2])

0.021102964412420988

In [6]:
PPI_interactome_tools.get_ppi_prob("Q9H9S0", "Q01860")

In [7]:
210/0.4*26.5/1000

13.9125

In [8]:
x1 = 6165000
x3 = 210217000 - 2151000
x2 = (x3 - x1) * 1/3 + x1
x2

73465333.33333333

In [9]:
from molytica_m.data_tools import create_PPI_dataset
from molytica_m.data_tools import alpha_fold_tools
from tqdm.notebook import tqdm
import torch.optim as optim
import torch.nn as nn
import numpy as np
import torch
import h5py
import os
import sqlite3
from molytica_m.ml.PPI_B_model import ProteinInteractionPredictor

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("cuda" if torch.cuda.is_available() else "cpu")
model = ProteinInteractionPredictor().to(device)
model.eval()

cuda


ProteinInteractionPredictor(
  (fc1): Linear(in_features=18640, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (gcn1): GCNConv(9, 128)
  (gcn2): GCNConv(128, 128)
  (fc_combined): Linear(in_features=768, out_features=128, bias=True)
  (output): Linear(in_features=128, out_features=1, bias=True)
)

In [10]:
import random
uniprot_pairs_batch = [[random.choice(af_uniprots), random.choice(af_uniprots)], [random.choice(af_uniprots), random.choice(af_uniprots)], [random.choice(af_uniprots), random.choice(af_uniprots)], [random.choice(af_uniprots), random.choice(af_uniprots)]]

In [12]:
from torch_geometric.data import Data, Batch
def create_batch(uniprot_pairs_batch):
    metadata_as = []
    metadata_bs = []

    data_as = []
    data_bs = []

    for uniprot_pair in uniprot_pairs_batch:
        uniprot_A = uniprot_pair[0]
        uniprot_B = uniprot_pair[1]

        metadata_as.append(create_PPI_dataset.get_metadata(uniprot_A))
        metadata_bs.append(create_PPI_dataset.get_metadata(uniprot_B))
        
        x_a, edge_index_a = create_PPI_dataset.get_graph_raw(uniprot_A)
        x_b, edge_index_b = create_PPI_dataset.get_graph_raw(uniprot_B)

        data_as.append(Data(x=torch.Tensor(x_a), edge_index=torch.tensor(edge_index_a, dtype=torch.long)))
        data_bs.append(Data(x=torch.Tensor(x_b), edge_index=torch.tensor(edge_index_b, dtype=torch.long)))

    metadata_as = np.array(metadata_as)
    metadata_bs = np.array(metadata_bs)
    metadata_as = torch.tensor(metadata_as, dtype=torch.float).to(device)
    metadata_bs = torch.tensor(metadata_bs, dtype=torch.float).to(device)

    batch_as = Batch.from_data_list(data_as).to(device)
    batch_bs = Batch.from_data_list(data_bs).to(device)
    return metadata_as, metadata_bs, batch_as, batch_bs

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from molytica_m.ml.PPI_B_model import ProteinInteractionPredictor
model = ProteinInteractionPredictor().to(device)
loss_function = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [13]:
metadata_as, metadata_bs, batch_as, batch_bs = create_batch(uniprot_pairs_batch)
model(metadata_as, metadata_bs, batch_as.x, batch_as.edge_index, batch_as.batch, batch_bs.x, batch_bs.edge_index, batch_bs.batch)

tensor([[0.4812],
        [0.4815],
        [0.4811],
        [0.4809]], device='cuda:0', grad_fn=<SigmoidBackward0>)

In [7]:
from molytica_m.data_tools.graph_tools import interactome_tools
combined_edge_list = interactome_tools.get_full_edge_list()

In [16]:
from tqdm.notebook import tqdm
import random
from molytica_m.data_tools import alpha_fold_tools
from molytica_m.data_tools import create_PPI_dataset
import numpy as np
af_uniprots = alpha_fold_tools.get_alphafold_uniprot_ids()
batch_size = 50

l_comb_edge_list = len(combined_edge_list)
for idx in tqdm(range(0, len(combined_edge_list), batch_size), desc="Training"):
    batch = []
    labels = []
    for x in range(min(batch_size, l_comb_edge_list - idx)):
        batch.append(combined_edge_list[idx + x])
        labels.append(1)

        # Add negative example
        batch.append([random.choice(af_uniprots), random.choice(af_uniprots)])
        labels.append(0)

    metadata_as, metadata_bs, batch_as, batch_bs = create_batch(batch)

    outputs = model(metadata_as, metadata_bs, batch_as.x, batch_as.edge_index, batch_as.batch, batch_bs.x, batch_bs.edge_index, batch_bs.batch)
    labels = torch.tensor(np.array(labels), dtype=torch.float).unsqueeze(1).to(device)

    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()

Training:   0%|          | 0/15800 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.21 GiB. GPU 0 has a total capacty of 7.79 GiB of which 230.19 MiB is free. Including non-PyTorch memory, this process has 7.55 GiB memory in use. Of the allocated memory 5.51 GiB is allocated by PyTorch, and 1.85 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

device(type='cpu')

In [3]:
import torch
torch.cuda.is_available()


True

In [3]:
import os

["https://ftp.ebi.ac.uk/pub/databases/alphafold/latest/" + x for x in os.listdir("data/curated_chembl/all_alpha_fold_data/")]

['https://ftp.ebi.ac.uk/pub/databases/alphafold/latest/UP000001450_36329_PLAF7_v4.tar',
 'https://ftp.ebi.ac.uk/pub/databases/alphafold/latest/UP000001014_99287_SALTY_v4.tar',
 'https://ftp.ebi.ac.uk/pub/databases/alphafold/latest/UP000024404_6282_ONCVO_v4.tar',
 'https://ftp.ebi.ac.uk/pub/databases/alphafold/latest/UP000002296_353153_TRYCC_v4.tar',
 'https://ftp.ebi.ac.uk/pub/databases/alphafold/latest/UP000006672_6279_BRUMA_v4.tar',
 'https://ftp.ebi.ac.uk/pub/databases/alphafold/latest/UP000002311_559292_YEAST_v4.tar',
 'https://ftp.ebi.ac.uk/pub/databases/alphafold/latest/UP000007841_1125630_KLEPH_v4.tar',
 'https://ftp.ebi.ac.uk/pub/databases/alphafold/latest/UP000008816_93061_STAA8_v4.tar',
 'https://ftp.ebi.ac.uk/pub/databases/alphafold/latest/UP000008153_5671_LEIIN_v4.tar',
 'https://ftp.ebi.ac.uk/pub/databases/alphafold/latest/UP000000589_10090_MOUSE_v4.tar',
 'https://ftp.ebi.ac.uk/pub/databases/alphafold/latest/UP000000803_7227_DROME_v4.tar',
 'https://ftp.ebi.ac.uk/pub/data